In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from notebook_utils import load_results, plot_results, load_all_results, load_results_for_one_checkpoint

In [3]:
%%time
results = load_all_results()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10854/10854 [04:23<00:00, 41.23it/s]

CPU times: user 3min 22s, sys: 3.93 s, total: 3min 26s
Wall time: 4min 23s


In [11]:
from glob import glob
results_mbert = load_results_for_one_checkpoint("finetune/bert-base-multilingual-cased", "bert-base-multilingual-cased", 0)

In [12]:
all_results = results+results_mbert

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(all_results)

In [15]:
average_df = df[df.lang == "cs-uk"]
average_df = average_df.drop(columns="lang").groupby(["experiment_name", "step"]).mean().reset_index()
best_steps = average_df.loc[average_df.groupby('experiment_name').aer.idxmin()]
best_steps

,experiment_name,step,aer,precision,recall,f1
0,bert-base-multilingual-cased,0,0.125252,0.855440,0.897458,0.875945
23,mbert_full8M_1epoch,16000,0.094378,0.895833,0.916740,0.906166
104,mbert_full8M_1epoch_finegrained_bs16,5200,0.095373,0.888889,0.922875,0.905563
158,mbert_multilingual_1M-per-lang,4000,0.092721,0.893778,0.922875,0.908094
905,mbert_multilingual_1M-per-lang_only_tlm,333000,0.106011,0.880394,0.909728,0.894821
912,mbert_multilingual_1M-per-lang_only_tlm_add_so,1200,0.089910,0.899013,0.922875,0.910788
1037,mbert_multilingual_1M-per-lang_only_tlm_add_so...,6200,0.088871,0.899390,0.924628,0.911834
1149,mbert_multilingual_1M-per-lang_only_tlm_add_so...,8600,0.088211,0.899166,0.926380,0.912570


In [16]:
allowed = list(best_steps[["experiment_name", "step"]].apply(tuple, axis=1))
condition = df[['experiment_name', 'step']].apply(tuple, axis=1).isin(allowed)
p = df[condition].pivot(index=("experiment_name", "step"), columns="lang", values="aer")*100
p.style.highlight_min(color='lightgreen', axis = 0).format('{:.1f}')

,lang,cs-uk,de-en,en-cs,en-fr,es-fr,ja-en,ro-en,zh-en
experiment_name,step,,,,,,,,
bert-base-multilingual-cased,0,12.5,17.4,9.5,5.6,14.4,45.6,27.9,17.9
mbert_full8M_1epoch,16000,9.4,16.1,7.3,5.1,15.3,47.8,28.0,16.3
mbert_full8M_1epoch_finegrained_bs16,5200,9.5,15.5,6.9,4.4,14.3,44.8,25.6,15.3
mbert_multilingual_1M-per-lang,4000,9.3,15.3,6.3,4.8,14.3,44.7,25.8,14.9
mbert_multilingual_1M-per-lang_only_tlm,333000,10.6,16.0,7.5,4.3,13.9,47.6,39.4,17.0
mbert_multilingual_1M-per-lang_only_tlm_add_so,1200,9.0,14.6,6.1,3.9,13.8,46.0,33.5,14.8
mbert_multilingual_1M-per-lang_only_tlm_add_so_lr1e-5,6200,8.9,14.4,6.0,3.9,13.8,45.9,32.4,14.5
mbert_multilingual_1M-per-lang_only_tlm_add_so_lr5e-6,8600,8.8,14.4,5.8,3.9,13.7,46.0,32.8,14.6
